# Analyzing IMDB Data in Keras - Solution

In [79]:
# Imports
import numpy as np
import keras
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.preprocessing.text import Tokenizer
import matplotlib.pyplot as plt
%matplotlib inline

np.random.seed(42)

## 1. Loading the data
This dataset comes preloaded with Keras, so one simple command will get us training and testing data. There is a parameter for how many words we want to look at. We've set it at 1000, but feel free to experiment.

In [80]:
# Loading the data (it's preloaded in Keras)
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=1000, skip_top=10)

print(x_train.shape)
print(x_test.shape)
print(x_train[857])
print(y_train[857])
print(len(x_train[857]))

(25000,)
(25000,)
[2, 32, 11, 32, 2, 52, 22, 2, 128, 18, 2, 192, 15, 69, 2, 22, 24, 77, 93, 2, 65, 238, 2, 2, 2, 2, 2, 2, 127, 2, 52, 292, 17, 2, 2, 298, 19, 2, 2, 501, 2, 2, 2, 2, 2, 2, 49, 14, 203, 30, 99, 2, 2, 409, 357, 18, 72, 12, 2, 2, 2, 73, 579, 787, 2, 404, 18, 223, 829, 150, 198, 24, 142, 25, 79, 2, 176, 2, 2]
1
79


## 2. Examining the data
Notice that the data has been already pre-processed, where all the words have numbers, and the reviews come in as a vector with the words that the review contains. For example, if the word 'the' is the first one in our dictionary, and a review contains the word 'the', then there is a 1 in the corresponding vector.

The output comes as a vector of 1's and 0's, where 1 is a positive sentiment for the review, and 0 is negative.

In [81]:
print(x_train[0])
print(y_train[0])

[2, 14, 22, 16, 43, 530, 973, 2, 2, 65, 458, 2, 66, 2, 2, 173, 36, 256, 2, 25, 100, 43, 838, 112, 50, 670, 2, 2, 35, 480, 284, 2, 150, 2, 172, 112, 167, 2, 336, 385, 39, 2, 172, 2, 2, 17, 546, 38, 13, 447, 2, 192, 50, 16, 2, 147, 2, 19, 14, 22, 2, 2, 2, 469, 2, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 2, 2, 22, 17, 515, 17, 12, 16, 626, 18, 2, 2, 62, 386, 12, 2, 316, 2, 106, 2, 2, 2, 2, 16, 480, 66, 2, 33, 2, 130, 12, 16, 38, 619, 2, 25, 124, 51, 36, 135, 48, 25, 2, 33, 2, 22, 12, 215, 28, 77, 52, 2, 14, 407, 16, 82, 2, 2, 2, 107, 117, 2, 15, 256, 2, 2, 2, 2, 2, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 2, 2, 2, 2, 13, 104, 88, 2, 381, 15, 297, 98, 32, 2, 56, 26, 141, 2, 194, 2, 18, 2, 226, 22, 21, 134, 476, 26, 480, 2, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 2, 226, 65, 16, 38, 2, 88, 12, 16, 283, 2, 16, 2, 113, 103, 32, 15, 16, 2, 19, 178, 32]
1


## 3. One-hot encoding the output
Here, we'll turn the input vectors into (0,1)-vectors. For example, if the pre-processed vector contains the number 14, then in the processed vector, the 14th entry will be 1.

In [82]:
# Turning the output into vector mode, each of length 1000
tokenizer = Tokenizer(num_words=1000)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')
print(x_train.shape)
print(x_test.shape)
#print(x_train[0])

(25000, 1000)
(25000, 1000)


And we'll one-hot encode the output.

In [83]:
# One-hot encoding the output
num_classes = 2
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(y_train.shape)
print(y_test.shape)

(25000, 2)
(25000, 2)


## 4. Building a model - a wide one, two levels.


In [84]:
# Building the model architecture with one layer of length 100
model = Sequential()
model.add(Dense(1024, activation='relu', input_dim=1000))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

# Compiling the model using categorical_crossentropy loss, and rmsprop optimizer.
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_34 (Dense)             (None, 1024)              1025024   
_________________________________________________________________
dense_35 (Dense)             (None, 512)               524800    
_________________________________________________________________
dropout_33 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_36 (Dense)             (None, 2)                 1026      
Total params: 1,550,850.0
Trainable params: 1,550,850.0
Non-trainable params: 0.0
_________________________________________________________________


## 5. Training the model
Run the model here. Experiment with different batch_size, and number of epochs!

In [85]:
# Running and evaluating the model
from keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(filepath='saved_models/weights.wideModel.hdf5', 
                               verbose=1, save_best_only=True)

hist = model.fit(x_train, y_train,
          batch_size=32,
          epochs=10,
          validation_data=(x_test, y_test), callbacks=[checkpointer],
          verbose=2)

Train on 25000 samples, validate on 25000 samples
Epoch 1/10


ResourceExhaustedError: OOM when allocating tensor with shape[1024,512]
	 [[Node: mul_191 = Mul[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/gpu:0"](lr_9/read, gradients_7/dense_35/MatMul_grad/MatMul_1)]]

Caused by op 'mul_191', defined at:
  File "/local/mnt/workspace/vharavu/anaconda3/envs/dog-project/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/local/mnt/workspace/vharavu/anaconda3/envs/dog-project/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/local/mnt/workspace/vharavu/anaconda3/envs/dog-project/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/local/mnt/workspace/vharavu/anaconda3/envs/dog-project/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/local/mnt/workspace/vharavu/anaconda3/envs/dog-project/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477

## 6. Evaluating the model


In [ ]:
model.load_weights('saved_models/weights.wideModel.hdf5')
score = model.evaluate(x_test, y_test, verbose=0)
print("accuracy: ", score[1])

## 7. Build a deep (deeper than the previous wide model) model

In [ ]:
# Building the model architecture with three layers
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=1000))
model.add(Dropout(0.2))
model.add(Dense(80, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(60, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

# Compiling the model using categorical_crossentropy loss, and rmsprop optimizer.
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath='saved_models/weights.deepModel.hdf5',
                               save_best_only=True)

model.fit(x_train, y_train,
          batch_size=32,
          epochs=10,
          validation_data=(x_test, y_test), callbacks=[checkpointer],
          verbose=2)

model.load_weights('saved_models/weights.deepModel.hdf5')
score = model.evaluate(x_test, y_test, verbose=0)
print("accuracy: ", score[1])



## 8. Create a CNN

In [ ]:
'''from keras.layers import Conv2D, MaxPooling2D, MaxPooling1D, Flatten
modelC = Sequential()
print(x_train.shape)
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1, 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1, 1)
print(x_train.shape[1:])
modelC.add(Conv2D(32, (1, 1), activation='relu', input_shape=x_train.shape[1:]))
modelC.add(Conv2D(32, (1, 1), activation='relu'))
modelC.add(MaxPooling1D(pool_size=2))
modelC.add(Dropout(0.25))

modelC.add(Flatten())
modelC.add(Dense(256, activation='relu'))
modelC.add(Dropout(0.5))
modelC.add(Dense(num_classes, activation='softmax'))
modelC.summary()

modelC.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath='saved_models/weights.cnnModel.hdf5',
                               save_best_only=True)

modelC.fit(x_train, y_train,
          batch_size=32,
          epochs=10,
          validation_data=(x_test, y_test), callbacks=[checkpointer],
          verbose=2)

modelC.load_weights('saved_models/weights.cnnModel.hdf5')
score = modelC.evaluate(x_test, y_test, verbose=0)
print("accuracy: ", score[1])
'''
